# Graduates Admission - Responsible AI

In this notebook, we will use Microsoft's 'Responsible AI' library

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns


In [ ]:
data = pd.read_csv('./data/admission_data-rai.csv')
data.head()


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

data_for_shap = data.copy()
data_for_shap['Gender'].replace({'M': 1, 'F': 0}, inplace=True)
X = data_for_shap.drop(columns=['Chance of Admit'])
y = data_for_shap['Chance of Admit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
rf = RandomForestRegressor().fit(X_train, y_train)
print(f"Root mean squared test error = {np.sqrt(np.mean((rf.predict(X_test) - y_test)**2))}")


In [ ]:
train = X_train.copy()
train['Chance of Admit'] = y_train

test = X_test.copy()
test['Chance of Admit'] = y_test


In [ ]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights



In [ ]:
task_type = 'regression'
categorical_features = ['University Rating', 'Research', 'Gender']
rai_insights = RAIInsights(rf, train, test, 'Chance of Admit', task_type, categorical_features=categorical_features)


In [ ]:
rai_insights.explainer.add()
rai_insights.error_analysis.add()
rai_insights.causal.add(treatment_features=['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR', 'CGPA', 'Research'])
rai_insights.counterfactual.add(total_CFs=10, desired_range=[0.8, 1.0])


In [ ]:
rai_insights.compute()


In [ ]:
ResponsibleAIDashboard(rai_insights, port=5002)
